### Load Mujoco Engine

In [1]:
import mujoco
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append("../../")
from utils.mujoco_parser import MuJoCoParserClass
from utils.util import sample_xyzs,rpy2r,r2quat
np.set_printoptions(precision=2,suppress=True,linewidth=100)
plt.rc('xtick',labelsize=6); plt.rc('ytick',labelsize=6)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
print ("MuJoCo version:[%s]"%(mujoco.__version__))

MuJoCo version:[3.1.4]


### Parse

In [3]:
xml_path = '../../asset/ur5e/scene_ur5e_rg2_obj.xml'
env = MuJoCoParserClass(name='UR5e with RG2 gripper',rel_xml_path=xml_path,VERBOSE=True)
obj_names = [body_name for body_name in env.body_names
             if body_name is not None and (body_name.startswith("obj_"))]
n_obj = len(obj_names)
# Place objects in a row
xyzs = sample_xyzs(n_sample=n_obj,
                   x_range=[0.45,1.65],y_range=[-0.38,0.38],z_range=[0.81,0.81],min_dist=0.2)
colors = np.array([plt.cm.gist_rainbow(x) for x in np.linspace(0,1,n_obj)])
for obj_idx,obj_name in enumerate(obj_names):
    jntadr = env.model.body(obj_name).jntadr[0]
    env.model.joint(jntadr).qpos0[:3] = xyzs[obj_idx,:]
    geomadr = env.model.body(obj_name).geomadr[0]
    env.model.geom(geomadr).rgba = colors[obj_idx] # color

# Move tables and robot base
env.model.body('base_table').pos = np.array([0,0,0])
env.model.body('front_object_table').pos = np.array([1.05,0,0])
env.model.body('side_object_table').pos = np.array([0,-0.85,0])
env.model.body('base').pos = np.array([0,0,0.8])
print ("Ready.")

dt:[0.0020] HZ:[500]
n_dof (=nv):[60]
n_geom:[52]
geom_names:['floor', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
n_body:[29]
body_names:['world', 'base', 'shoulder_link', 'upper_arm_link', 'forearm_link', 'wrist_1_link', 'wrist_2_link', 'wrist_3_link', 'tcp_link', 'camera_mount', 'rg2_gripper_base_link', 'camera_center', 'rg2_gripper_finger1_finger_link', 'rg2_gripper_finger1_inner_knuckle_link', 'rg2_gripper_finger1_finger_tip_link', 'rg2_gripper_finger2_finger_link', 'rg2_gripper_finger2_inner_knuckle_link', 'rg2_gripper_finger2_finger_tip_link', 'front_object_table', 'side_object_table', 'base_table', 'obj_cylinder_01', 'obj_cylinder_02', 'obj_cylinder_03', 'obj_cylinder_04', 'obj_cylinder_05', 'obj_cylinder_06', 'obj_cylin

### Get gripper joint indexes

In [4]:
joint_names = env.rev_joint_names[:6]
idxs_forward = [env.model.joint(joint_name).qposadr[0] for joint_name in env.joint_names[:6]]
idxs_jacobian = [env.model.joint(joint_name).dofadr[0] for joint_name in env.joint_names[:6]]
list1, list2 = env.ctrl_joint_idxs, idxs_forward
idxs_step = []
for i in range(len(list2)):
    if list2[i] not in list1:
        idxs_step.append(list1.index(list2[i]))
        
gripper_joint_idxs = []

for gripper_idx in env.rev_joint_idxs:
    if gripper_idx in env.idxs_forward:
        gripper_joint_idxs.append(gripper_idx)
print(f"Gripper joint idxs: {gripper_joint_idxs}")

Gripper joint idxs: [0, 1, 2, 3, 4, 5]


In [5]:
init_q = [-1.570521656666891, -2.311883111993307, 2.1441715399371546, 1.73920385419812, 0.7853929996490479, -1.5711215178119105]
q_close_grasp = [-0.03, -0.0273, -0.03, 0.03, 0.0273, 0.03]

## Repel Inverse Kinematic

In [17]:
env.init_viewer(viewer_title='UR5e with RG2 gripper',viewer_width=1200,viewer_height=800,
                viewer_hide_menus=True)
env.update_viewer(azimuth=0.0,distance=3.5,elevation=-60,lookat=[0.4,0.05,0.36],
                  VIS_TRANSPARENT=False,VIS_CONTACTPOINT=True,
                  contactwidth=0.05,contactheight=0.05,contactrgba=np.array([1,0,0,1]),
                  VIS_JOINT=True,jointlength=0.5,jointwidth=0.1,jointrgba=[0.2,0.6,0.8,0.6])
env.reset()

p_trgt=np.array([0.7, -0.2, 0.9])
R_trgt = rpy2r(np.radians([-180,0,90]))
q_init=np.array(init_q)
body_name='tcp_link'

render_every = 1
th=1*np.pi/180.0
err_th=1e-3
DO_RENDER = True

all_body_names = env.get_body_names(prefix='')
prefix_collision = 'base_table'
self_collision = ''

q_backup = env.get_q(joint_idxs=env.idxs_forward)
q = q_init.copy()
env.forward(q=q,joint_idxs=env.idxs_forward)
tick = 0

while True:
    tick = tick + 1
    J,err = env.get_ik_ingredients(
        body_name=body_name,p_trgt=p_trgt,R_trgt=R_trgt,IK_P=True,IK_R=True)
    dq = env.damped_ls(J,err,stepsize=1,eps=1e-1,th=th)
    q = q + dq[env.idxs_jacobian]

    env.forward(q=q,joint_idxs=env.idxs_forward)

    p_contacts,f_contacts,geom1s,geom2s,body1s,body2s = env.get_contact_info(must_include_prefix=None,must_exclude_prefix=None)
    # p_contacts,f_contacts,geom1s,geom2s = env.get_multiple_contact_info(must_include_prefixes=None, must_exclude_prefixes='rg2_')
    print(f"p_contacts: {p_contacts}")
    print(f"f_contacts: {f_contacts}")
    print(f"geom1s: {geom1s}")
    print(f"geom2s: {geom2s}")

    while len(geom1s) > 0 and len(geom2s) > 0:
        print("Collision detected!")
        # revert to previous state. (Before collision.)
        q = q - dq[env.idxs_jacobian]

        for contact_geom_idx, (geom1, geom2) in enumerate(zip(geom1s, geom2s)):
            print(f"Index: {contact_geom_idx}, geom1: {geom1}, geom2: {geom2}")

            if geom1 is None or geom2 is None:  # geom1이나 geom2가 None이면 Pass
                continue

            # Detect Self-collision.
            if geom1 not in all_body_names: # if the name of the robot's geom is not in the list of all body names
                print(f"{geom1} is not in all_body_names")
                cut_geom1_name = geom1.split('_')[0:-1] # slice all elements except the last one
                new_geom1_name = '_'.join(cut_geom1_name)     # join the sliced elements back into a string
                print(f"new geom1 name: {new_geom1_name}")

            # Detect with the Table.
            if geom2 not in all_body_names: # if the name of the robot's geom is not in the list of all body names
                print(f"{geom2} is not in all_body_names")
                cut_geom2_name = geom2.split('_')[0:-1] # slice all elements except the last one
                new_geom2_name = '_'.join(cut_geom2_name)     # join the sliced elements back into a string
                print(f"new geom2 name: {new_geom2_name}")

        # contact_body = new_geom1_name

        p_contact = p_contacts[contact_geom_idx]
        f_contact = f_contacts[contact_geom_idx]
        f_norm = np.linalg.norm(f_contact) # normalize the force
        repel_direction = f_contact / (f_norm+1e-8) # numerical prevention
        # env.plot_arrow(p=p_contact,uv=repel_direction,r_stem=0.01,len_arrow=0.05,rgba=[1,0,0,1], label='')
        env.plot_sphere(p=p_contact,r=0.02,rgba=[1,0.2,0.2,1],label='')
        for p in p_contacts: env.plot_sphere(p=p,r=0.05,rgba=[1,0.2,0.2,1],label='contact points')

        repel_step_size = 1
        repel_scale = 1.0
        p_repel_trgt = p_contact + repel_direction * repel_scale

        J_contact, err_contact = env.get_ik_ingredients(
        body_name=body_name,p_trgt=p_repel_trgt,R_trgt=np.eye(3),IK_P=True,IK_R=False)
        dq = env.damped_ls(J_contact, err_contact,stepsize=repel_step_size,eps=1e-1,th=th)
        q = q + dq[env.idxs_jacobian]
        env.forward(q=q,joint_idxs=env.idxs_forward)

        env.plot_T(p=p_contact,R=np.eye(3),PLOT_AXIS=True,axis_len=0.1,axis_width=0.005, label='p_contact')
        env.plot_T(p=p_repel_trgt,R=np.eye(3),PLOT_AXIS=True,axis_len=0.2,axis_width=0.005, label='p_repel_trgt')
        env.render()

        p_contacts,f_contacts,geom1s,geom2s,body1s,body2s = env.get_contact_info(must_include_prefix=None,must_exclude_prefix='rg2_')
        # p_contacts,f_contacts,geom1s,geom2s = env.get_multiple_contact_info(must_include_prefixes=None, must_exclude_prefixes='rg2_')

        
    env.forward(q=q,joint_idxs=env.idxs_forward)
    # Terminate condition
    err_norm = np.linalg.norm(err)
    if err_norm < err_th:
        break
    # Render
    if DO_RENDER:
        if ((tick-1)%render_every) == 0:
            p_tcp,R_tcp = env.get_pR_body(body_name=body_name)
            env.plot_T(p=p_tcp,R=R_tcp,PLOT_AXIS=True,axis_len=0.1,axis_width=0.005)
            env.plot_T(p=p_trgt,R=R_trgt,PLOT_AXIS=True,axis_len=0.2,axis_width=0.005)
            env.render()
# Back to back-uped position
q_ik = env.get_q(joint_idxs=env.idxs_forward)
env.forward(q=q_backup,joint_idxs=env.idxs_forward)

# Close viewer
env.close_viewer()
print ("Done.")

p_contacts: [array([ 0.33, -0.  ,  1.15])]
f_contacts: [array([ 0.04, -0.04, 67.62])]
geom1s: [None]
geom2s: [None]
Collision detected!
Index: 0, geom1: None, geom2: None
p_contacts: [array([ 0.33, -0.  ,  1.15])]
f_contacts: [array([-0.04, -0.04, 67.62])]
geom1s: [None]
geom2s: [None]
Collision detected!
Index: 0, geom1: None, geom2: None
p_contacts: [array([ 0.33, -0.  ,  1.16])]
f_contacts: [array([-0.1 , -0.04, 67.62])]
geom1s: [None]
geom2s: [None]
Collision detected!
Index: 0, geom1: None, geom2: None
p_contacts: [array([ 0.34, -0.  ,  1.17])]
f_contacts: [array([-0.16, -0.04, 67.62])]
geom1s: [None]
geom2s: [None]
Collision detected!
Index: 0, geom1: None, geom2: None
p_contacts: [array([ 0.34, -0.  ,  1.18])]
f_contacts: [array([-0.2 , -0.05, 67.62])]
geom1s: [None]
geom2s: [None]
Collision detected!
Index: 0, geom1: None, geom2: None
p_contacts: [array([ 0.34, -0.  ,  1.18])]
f_contacts: [array([-0.23, -0.05, 67.62])]
geom1s: [None]
geom2s: [None]
Collision detected!
Index: 0,

Exception: GLFW window does not exist but you tried to render.